In [1]:
from qdrant_client import QdrantClient, models
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=False, device="cpu")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/harddisk/projects/legal_ml/.venv/lib/python3.10/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = 

https://github.com/qdrant/workshop-ultimate-hybrid-search/blob/main/notebooks/02-hybrid-search.ipynb

### Создание БД с несколькими векторами

In [2]:
vectors = model.encode(
    "Как мне получить выплаты?",
    return_sparse=True,
    return_dense=True,
    return_colbert_vecs=True,
)
vectors

{'dense_vecs': array([-0.05599624,  0.02769903, -0.04615401, ..., -0.03377588,
        -0.02925543, -0.04329142], dtype=float32),
 'lexical_weights': defaultdict(int,
             {'5187': 0.13252598,
              '4042': 0.15637419,
              '28068': 0.2146028,
              '138417': 0.31710362,
              '32': 0.04425759}),
 'colbert_vecs': array([[-0.04191731,  0.02044222,  0.01853201, ...,  0.0173075 ,
          0.06125861, -0.01522963],
        [-0.03104483,  0.00796479,  0.0223996 , ...,  0.01070459,
          0.04036965,  0.00780059],
        [-0.03103829,  0.02579158,  0.03169292, ...,  0.0075931 ,
          0.04057872, -0.00449112],
        [-0.04662576,  0.0143568 ,  0.05201059, ...,  0.01773259,
          0.04036565,  0.00740162],
        [-0.0451145 ,  0.000516  ,  0.02601848, ...,  0.02121259,
          0.07327194,  0.0132298 ],
        [ 0.0091881 ,  0.00524948, -0.009853  , ...,  0.01237569,
          0.03225253,  0.02549963]], dtype=float32)}

In [3]:
vectors["dense_vecs"].shape, vectors["colbert_vecs"].shape, vectors["lexical_weights"]

((1024,),
 (6, 1024),
 defaultdict(int,
             {'5187': 0.13252598,
              '4042': 0.15637419,
              '28068': 0.2146028,
              '138417': 0.31710362,
              '32': 0.04425759}))

In [4]:
model.convert_id_to_token(
    [
        {
            "5187": 0.13655508,
            "4042": 0.15718421,
            "28068": 0.1956503,
            "207964": 0.22945362,
            "138417": 0.25035983,
            "32": 0.034747005,
        }
    ]
)

{'Как': 0.13655508,
 'мне': 0.15718421,
 'получить': 0.1956503,
 'социальные': 0.22945362,
 'выплаты': 0.25035983,
 '?': 0.034747005}

In [5]:
client = QdrantClient(":memory:")

col_name = "hybrid_search_test"

In [6]:
client.create_collection(
    collection_name=col_name,
    vectors_config={
        "dence": models.VectorParams(
            size=vectors["dense_vecs"].shape[0], distance=models.Distance.COSINE
        ),
        "colbert": models.VectorParams(
            size=vectors["colbert_vecs"].shape[1],
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM
            ),
        ),
    },
    sparse_vectors_config={"text-sparse": models.SparseVectorParams()},
)

True

In [7]:
points = [
    models.PointStruct(
        id=3,
        vector={
            "dence": vectors["dense_vecs"],  # Плотный вектор
            "colbert": vectors["colbert_vecs"],  # Плотный вектор
            "text-sparse": models.SparseVector(
                indices=list(vectors["lexical_weights"].keys()),  # Индексы ненулевых элементов
                values=list(vectors["lexical_weights"].values()),  # Значения ненулевых элементов
            ),
        },
        payload={
            "text": "Пример текста 3",
            "metadata": {"author": "Автор 3"},
        },
    ),
]

client.upsert(
    collection_name=col_name,
    points=points,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
sent = ["Как мне получить выплаты?", "социальные службы"]
embeddings_question = model.encode(
    sent,
    return_sparse=True,
    return_dense=True,
    return_colbert_vecs=True,
)
batch = [{"question": i, "answer": i, "rating": i, "id": index} for index, i in enumerate(sent)]

In [9]:
embeddings_question["lexical_weights"][0].keys()

dict_keys(['5187', '4042', '28068', '138417', '32'])

In [10]:
points = [
    models.PointStruct(
        id=record["id"],
        payload={
            "question": record["question"],
            "answer": record["answer"],
            "rating": record["rating"],
        },
        vector={
            "dense": dence,
            "colbert": colbert,
            "text-sparse": models.SparseVector(
                indices=list(sparce.keys()),
                values=list(sparce.values()),
            ),
        },
    )
    for dence, sparce, colbert, record in zip(
        embeddings_question["dense_vecs"],
        embeddings_question["lexical_weights"],
        embeddings_question["colbert_vecs"],
        batch,
    )
]

In [83]:
qdrant_client = QdrantClient(url="localhost:6333")

In [84]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='questions2'), CollectionDescription(name='questions'), CollectionDescription(name='questions3'), CollectionDescription(name='911_hybrid')])

In [108]:
point = qdrant_client.query_points(
    collection_name="911_hybrid",
    query=models.SampleQuery(sample=models.Sample.RANDOM),
    limit=1,
    with_vectors=True
).points[0]

vector = point.vector
question = point.payload["question"]

In [114]:
vector["dense"], vector["text-sparse"], question

([-0.028905766,
  0.029821472,
  -0.05015013,
  0.010088021,
  0.01755102,
  0.04203088,
  0.012041526,
  0.026937,
  0.02492245,
  -0.023258917,
  -0.021122271,
  0.023442058,
  0.0008088731,
  0.021427507,
  0.02979095,
  0.006352706,
  0.031958118,
  -0.0686779,
  -0.061535403,
  0.0032831854,
  -0.012186512,
  0.008355811,
  -0.0004960071,
  -0.010446672,
  0.024769831,
  0.018955102,
  -0.031500265,
  0.015948536,
  0.008241349,
  -0.0051737353,
  -0.052317303,
  -0.064404614,
  0.014269742,
  -0.033331677,
  0.007524046,
  -0.027165927,
  0.029485714,
  0.009202839,
  -0.002728039,
  0.011667613,
  0.04428962,
  0.014140017,
  -0.005994055,
  -0.052805677,
  -0.0052004433,
  -0.023884648,
  0.0011713398,
  -0.026036557,
  -0.08302396,
  -0.04032156,
  -0.024540905,
  0.017245784,
  0.10646601,
  0.017154215,
  0.00703567,
  0.065991834,
  0.0069212066,
  -0.022282165,
  -0.036719784,
  -0.0117362905,
  -0.035529368,
  -0.049905945,
  -0.03812387,
  0.013560071,
  0.0052195205,
  

In [137]:
point = qdrant_client.query_points(
    collection_name="911_hybrid",
    query=models.SampleQuery(sample=models.Sample.RANDOM),
    limit=1,
    with_vectors=True
).points[0]

vector = point.vector
question = point.payload["question"]


sparse = models.Prefetch(
    query=models.SparseVector(
        indices=vector["text-sparse"].indices,
        values=vector["text-sparse"].values,
    ),
    using="text-sparse",
    limit=100
)

# Второй этап: переранжирование результатов с использованием плотных векторов
dense = models.Prefetch(
    query=vector["dense"],
    using="dense",
    limit=100
)

# Второй этап: переранжирование результатов с использованием плотных векторов
dence_sparse = models.Prefetch(
    prefetch=[dense],
    query=models.SparseVector(
        indices=vector["text-sparse"].indices,
        values=vector["text-sparse"].values,
    ),
    using="text-sparse",
    limit=100
)

sparce_dense = models.Prefetch(
    prefetch=[sparse],
    query=vector["dense"],
    using="dense",
    limit=100
)


## Запрашиваем данные
multistage_search_results = qdrant_client.query_points(
    collection_name="911_hybrid",
    prefetch=[sparse, dense],
    limit=200,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
).points[1:4]

dense_search_results = qdrant_client.query_points(
    collection_name="911_hybrid",
    prefetch=[dense],
    limit=200,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
).points[1:4]

sparse_search_results = qdrant_client.query_points(
    collection_name="911_hybrid",
    prefetch=[sparse],
    limit=200,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
).points[1:4]

sparce_dense_search_results = qdrant_client.query_points(
    collection_name="911_hybrid",
    prefetch=[sparce_dense],
    limit=200,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
).points[1:4]

dence_sparse_search_results = qdrant_client.query_points(
    collection_name="911_hybrid",
    prefetch=[dence_sparse],
    limit=200,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
).points[1:4]


print(question)
print()
print("Комбинированный")
for index, i in enumerate(multistage_search_results):
    print("index", index)
    print(i.payload["question"])

print()
print("Плотный")
for index, i in enumerate(dense_search_results):
    print("index", index)
    print(i.payload["question"])


print()
print("Разряженный")
for index, i in enumerate(sparse_search_results):
    print("index", index)
    print(i.payload["question"])

print()
print("Разряженный -> Плотный")
for index, i in enumerate(sparce_dense_search_results):
    print("index", index)
    print(i.payload["question"])

print()
print("Плотный -> Разряженный")
for index, i in enumerate(dence_sparse_search_results):
    print("index", index)
    print(i.payload["question"])

Была продана дача 5 лет назад. Документы не оформлялись. Была выдана генеральная доверенность покупателю на совершение сделки. Но покупатель так и не удосужился оформить сделку и документы. Дача в собственности у продавца, но покупатель пользовался ей.Продавец понес из за этого некоторые убытки. Продавец теперь не хочет оформлять документы и может ли отнять дачу? Стоит ли обращаться в суд или уже прошел срок исковой давности?

Комбинированный
index 0
Если была продана дача 5 лет назад, но документы не оформляли, а была выдана генеральная доверенность на оформление сделки. Покупатель так и не оформил право собственности.  Продавец хочет забрать дачу обратно, должен он вернуть деньги? Прошло 5 лет.
index 1
Мама с отчимом прожила 9 лет, он умер. У него была дача, вместе ухаживали, и сейчас ухаживаем уже 24 года. Хотим оформить на маму. Родствеников у него нет, жили просто в гражданском браке, на одном предприятие работали, маму знают ей сами предложили оформить дачу. Мы обратились к знако

In [118]:
multistage_search_results

[ScoredPoint(id=12527648, version=26166, score=1.0, payload={'question': 'Муж устроился на работу в середине марта. За март в расчетке дети не учитываются в ндфл.  За апрель тоже самое. С какого месяца должны начать высчитывать подоходный на детей?', 'answer': 'Налоговый вычет предоставляется родителям, супругу на основании их письменных заявлений и документов, подтверждающих право на данный налоговый вычет. Устроившись на работу Вам надо было заявить о своих правах на налоговый вычет в письменном виде.', 'rating': 0.0}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12308630, version=10722, score=0.33333334, payload={'question': 'Что возможно сделать в моей ситуации. В июле 2016 года мы с бывшим супругом развелись, со мной остались дети 3-х и 5-ти лет. В апреле 2017 г. мне предстоит выходить на работу вахтовым методом  на 2 мес. (работа находится за полярным кругом). Детей оставляю на свою родную сестру. Муж узнав об этом по телефону грозится вывезти детей в мое отсут

In [134]:
point = qdrant_client.query_points(
    collection_name="911_hybrid",
    query=models.SampleQuery(sample=models.Sample.RANDOM),
    limit=1,
    with_vectors=True
).points[0]

vector = point.vector
question = point.payload["question"]

# Второй этап: переранжирование результатов с использованием плотных векторов
dense_rerank = models.Prefetch(
    query=vector["dense"],
    using="dense",
    limit=100
)

sparse_search = models.Prefetch(
    prefetch=[dense_rerank],
    query=models.SparseVector(
        indices=vector["text-sparse"].indices,
        values=vector["text-sparse"].values,
    ),
    using="text-sparse",
    limit=100
)


multistage_search_results = qdrant_client.query_points(
    collection_name="911_hybrid",
    prefetch=[sparse_search],
    limit=200,
    query=models.FusionQuery(
            fusion=models.Fusion.RRF,
        )
).points[1:4]


print(question)
print()
print("Комбинированный")
for index, i in enumerate(multistage_search_results):
    print("index", index)
    print(i.payload["question"])

Продаю комнату в коммунальной квартире. Один сосед-собственник, у другого-документы сданы на приватизацию. Моя комнату покупает второй. Как легче и быстрее всего взять отказ  у соседа - собственника (он может отказ оформить у нотариуса) и сколько это стоит?

Комбинированный
index 0
У нас с мамой две комнаты в многокомнатной коммунальной квартире (бывшее общежитие). Равные доли – по 15/563. Два свидетельства о регистрации.
 Я хочу продать свою комнату (долю). В квартире 27 сособственников, многие из них не прописаны и сдают свои комнаты. Рыночная стоимость комнаты около 800 т.р.
Хочу выбрать наименее затратный вариант продажи.
1. С нотариусом. А) Нотариус рассылает письма-извещения всем сособственникам. Ждем месяц, получаю заверенный отказ, ищу покупателя. Нахожу, составляем ДКП, сдаю на регистрацию.
Проблемы: неизвестен адрес регистрации половины собственников, придется рассылать на адреса нахождения недвижимости. Не завернет ли Росреестр такую сделку? + возможно есть несовершеннолетни